In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [3]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [4]:
dfAdmDiff = dfAdm.diff().iloc[1:]
plotDates = curDates[1:]

In [5]:
fig,ax1 = plt.subplots(1,1,sharex=True)

# ax1.plot(plotDates,dfAdmDiff['0-9'])
# ax1.plot(plotDates,dfAdmDiff['10-19'])
ax1.plot(plotDates,dfAdmDiff['0-9']+dfAdmDiff['10-19'],label='0-19')
ax1.plot(plotDates,dfAdmDiff['20-29'],label='20-29')
ax1.plot(plotDates,dfAdmDiff['30-39'],label='30-39')
ax1.plot(plotDates,dfAdmDiff['40-49'],label='40-49')

ax1.set_ylim(bottom=0)
ax1.set_xlim([plotDates[0],plotDates[-1]])

ax1.grid()
ax1.legend()
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
dfCase = pd.DataFrame(columns=dfAdm.columns)
dfTest = pd.DataFrame(columns=dfAdm.columns)
dfPosP = pd.DataFrame(columns=dfAdm.columns)

In [7]:
# Collect the cases, tests and positive percentage from all the "cases by age" files

ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        
        curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))
        
        
        dfCase.loc[latestDate] = curdf['Antal_bekræftede_COVID-19'].values
        dfTest.loc[latestDate] = curdf['Antal_testede'].values
        dfPosP.loc[latestDate] = curdf['Procent_positive'].values

allDates = dfCase.index


In [8]:
dfCaseDiff = dfCase.diff().iloc[1:]
dfTestDiff = dfTest.diff().iloc[1:]
dfPosPDiff = dfPosP.diff().iloc[1:]
plotDatesCase = allDates[1:]

In [9]:
fig,ax1 = plt.subplots(1,1)

howManyToUse = len(dfCaseDiff.columns)-4
cmap = plt.cm.get_cmap('Dark2',howManyToUse)
for k in range(0,howManyToUse):
    curColName = dfCaseDiff.columns[k]
    ax1.plot(plotDatesCase,dfCaseDiff[curColName],'.-',linewidth=0.4,label=curColName,color=cmap(k))
    # meanWidth = 7
    # ax1.plot(rnTime(plotDatesCase,meanWidth),rnMean(dfCaseDiff[curColName],meanWidth),label=curColName,color=cmap(k))

ax1.legend()

ax1.set_ylim(bottom=0,top=1000)
ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-05-05')])

plt.tight_layout()

# ax1.plot(plotDatesCase,dfCaseDiff['0-9'])
# ax1.plot(plotDatesCase,dfCaseDiff['10-19'])
# ax1.plot(plotDatesCase,dfCaseDiff['20-29'])
# ax1.plot(plotDatesCase,dfCaseDiff['30-39'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# dfCaseDiff

curIndex = dfCaseDiff.index.values.astype('datetime64[D]')
dayDiff = curIndex[1:] - curIndex[:-1]


In [11]:
fig,ax1 = plt.subplots(1,1)
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# for k in range(4,len(dayDiff)):
for k in range(0,len(dfCaseDiff)):

    
    # ax1.bar(curIndex[k]-dayDiff[k-1],dfCaseDiff['20-29'].iloc[k],width=dayDiff[k-1])
    curWidth = dayDiff[k-1]
    numDaysWidth = curWidth/np.timedelta64(1,'D')

    # Since half days aren't possible, convert to an "hours" timedelta64
    toMove = np.timedelta64(int(24*(numDaysWidth)/2),'h')

    curHeight = (dfCaseDiff['20-29'].iloc[k])/numDaysWidth
    # curHeight = (dfCaseDiff['30-39'].iloc[k])/numDaysWidth
    
    
    # ax1.bar(curIndex[k]+np.timedelta64(1,'D'),dfCaseDiff['20-29'].iloc[k],width=dayDiff[k-1])
    # ax1.bar(curIndex[k]-toMove,dfCaseDiff['20-29'].iloc[k],width=dayDiff[k-1])
    ax1.bar(curIndex[k]-toMove,curHeight,width=dayDiff[k-1],color='grey',edgecolor='k')
    
    # ax2.plot(curIndex[k],curHeight,'k*-')
    # ax2.plot(curIndex[k],dfCaseDiff['20-29'].iloc[k],'r.-')
    # ax1.plot(curIndex[k],dfCaseDiff['20-29'].iloc[k],'*')
    # ax2.plot(curIndex[k],dayDiff[k-1],'*')

ax1.set_xlim([np.datetime64('2020-11-01'),np.datetime64('2021-05-07')])
ax1.set_ylim(bottom=0)
# ax1.set_ylim(top=300)
# ax2.set_ylim(bottom=0)
# ax2.set_ylim(top=1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1025.6)

In [12]:
# dfCaseDiff.tail()

# plt.plot(allDays,allData,'.-')

In [13]:
# Make a daily dataframe
dfCaseFull = pd.DataFrame()
for curCol in dfCaseDiff.columns:
    allDays = []
    allData = []
    for k in range(0,len(dfCaseDiff)):

        
        curWidth = dayDiff[k-1]
        numDaysWidth = int(curWidth/np.timedelta64(1,'D'))
        
        # curHeight = (dfCaseDiff['20-29'].iloc[k])/numDaysWidth
        curHeight = (dfCaseDiff[curCol].iloc[k])/numDaysWidth

        curDay = curIndex[k]
        # print('----------')
        # print(curDay)
        # print(numDaysWidth)

        # for d in range(0,numDaysWidth):
        for d in range(numDaysWidth-1,-1,-1):
            dayToAdd = curDay - np.timedelta64(d,'D')
            # print(dayToAdd)
            allDays.append(dayToAdd)
            allData.append(curHeight)

    dfCaseFull['Dato']  = allDays
    # dfCaseFull['20-29'] = allData
    dfCaseFull[curCol] = allData


In [14]:
# plt.figure()
# plt.plot(allDays,allData,'.-')
# plt.show()
dfCaseFull.tail()
# dfCaseDiff.columns

,Dato,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
365,2021-05-07,56.000000,203.000000,261.0,155.0,147.000000,124.000000,55.000000,20.0,2.000000,0.0,1023.000000
366,2021-05-08,66.333333,178.333333,174.0,118.0,122.666667,104.333333,47.333333,14.0,2.666667,0.0,827.666667
367,2021-05-09,66.333333,178.333333,174.0,118.0,122.666667,104.333333,47.333333,14.0,2.666667,0.0,827.666667
368,2021-05-10,66.333333,178.333333,174.0,118.0,122.666667,104.333333,47.333333,14.0,2.666667,0.0,827.666667
369,2021-05-11,100.000000,196.000000,172.0,137.0,139.000000,100.000000,60.000000,14.0,6.000000,1.0,925.000000


In [15]:
colNames = dfCaseFull.columns[1:-1]
# numToShow = len(dfCaseFull.columns)-1
numToShow = len(dfCaseFull.columns)-2
fig,allAxes = plt.subplots(numToShow,1,sharex=True,figsize=(12,20))

meanWidth = 7


# allAxes[0].plot(dfCaseFull.Dato,dfCaseFull['0-9'],'.-')
# allAxes[0].plot(rnTime(dfCaseFull.Dato,meanWidth),rnMean(dfCaseFull['0-9'],meanWidth))
# allAxes[1].plot(dfCaseFull.Dato,dfCaseFull['10-19'],'.-')
# allAxes[1].plot(rnTime(dfCaseFull.Dato,meanWidth),rnMean(dfCaseFull['10-19'],meanWidth))

firstDate = np.datetime64('2021-02-01')
firstDateIndex = np.where(dfCaseFull.Dato == firstDate)[0][0]



# for ax in allAxes:
for k in range(0,numToShow):
    ax = allAxes[k]
    
    # ax.plot(dfCaseFull.Dato,dfCaseFull[colNames[k]],'.-')
    # ax.plot(rnTime(dfCaseFull.Dato,meanWidth),rnMean(dfCaseFull[colNames[k]],meanWidth)) 

    ax.plot(dfCaseFull.Dato.iloc[firstDateIndex:],dfCaseFull[colNames[k]].iloc[firstDateIndex:],'k.:',markersize=3,linewidth=0.5)
    ax.plot(rnTime(dfCaseFull.Dato.iloc[firstDateIndex:],meanWidth),rnMean(dfCaseFull[colNames[k]].iloc[firstDateIndex:],meanWidth),'k') 

    ax.set_title(colNames[k])

    ax.set_ylim(bottom=0)
    ax.grid()

# allAxes[0].set_xlim(left=np.datetime64('2020-11-01'))
allAxes[0].set_xlim(left=firstDate)

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesPerDay_AllAges')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Calculate the populations counts
dfPop = pd.read_excel('DKfolketal2021clean.xlsx') 

size1 = dfPop[0:20].Antal.sum()
size2 = dfPop[20:40].Antal.sum()
size3 = dfPop[40:70].Antal.sum()
size4 = dfPop[70:].Antal.sum()

sizes = [size1,size2,size3,size4]

In [17]:

# colNames = dfCaseFull.columns[1:-1]
# numToShow = len(dfCaseFull.columns)-1
# numToShow = len(dfCaseFull.columns)-2

group4 = dfCaseFull['90+']+dfCaseFull['80-89']+dfCaseFull['70-79']
group3 = dfCaseFull['40-49']+dfCaseFull['50-59']+dfCaseFull['60-69']
group2 = dfCaseFull['20-29']+dfCaseFull['30-39']
group1 = dfCaseFull['0-9']+dfCaseFull['10-19']

groupNames = ['0-19','20-39','40-69','70+']

groups = [group1,group2,group3,group4]
numToShow = len(groups)

# fig,allAxes = plt.subplots(numToShow,1,sharex=True,figsize=(12,20))
fig,ax1 = plt.subplots(1,1)

meanWidth = 7



# firstDate = np.datetime64('2021-02-01')
firstDate = np.datetime64('2020-10-01')
# firstDate = np.datetime64('2020-06-01')
# firstDate = dfCaseFull.Dato[0]
firstDateIndex = np.where(dfCaseFull.Dato == firstDate)[0][0]

curDates = dfCaseFull.Dato.iloc[firstDateIndex:]


# for ax in allAxes:

cmap = plt.cm.get_cmap('Dark2',numToShow+1)
for k in range(0,numToShow):

    curCases = groups[k].iloc[firstDateIndex:]
    curCasesNorm = np.divide(curCases,sizes[k])

    # ax1.plot(curDates,curCases,'.:',linewidth=0.5,markersize=3,color=cmap(k))
    # ax1.plot(rnTime(curDates,meanWidth),rnMean(curCases,meanWidth),color=cmap(k),label=groupNames[k]) 
    
    ax1.plot(curDates,10000 * curCasesNorm,'.:',linewidth=0.5,markersize=3,color=cmap(k))
    ax1.plot(rnTime(curDates,meanWidth),10000 * rnMean(curCasesNorm,meanWidth),color=cmap(k),label=groupNames[k]) 
    # ax1.plot(rnTime(dfCaseFull.Dato.iloc[firstDateIndex:],meanWidth),np.divide(rnMean(curCases,meanWidth),sizes[k]),color=cmap(k),label=groupNames[k]) 

ax1.set_ylim(bottom=0)
ax1.set_xlim(left=firstDate)

ax1.set_ylabel('Tilfælde per 10.000')

ax1.grid()
ax1.legend()
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesPerDay_AgeGroups')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
dfPop[90:]

,Alder,Antal
90,90,9971
91,91,8144
92,92,6919
93,93,5392
94,94,4405
95,95,3309
96,96,2480
97,97,1730
98,98,1101
99,99,806
